In [1]:
from datetime import timedelta
import os
import sys
sys.path.insert(0,'..')
import pandas as pd
from doe_xstock.database import SQLiteDatabase

In [2]:
DATABASE_FILEPATH = '../database.db'
DATABASE = SQLiteDatabase(DATABASE_FILEPATH)
DIRECTORY = '../data/ecobee'

In [ ]:
data_list = []

for f in os.listdir(DIRECTORY):
    if f.endswith('.parquet'):
        data = pd.read_parquet(os.path.join(DIRECTORY, f))
        data_list.append(data)
    else:
        pass

data = pd.concat(data_list, ignore_index=True, sort=False)
data = data.rename(columns={
    'Identifier':'building_name',
    'TemperatureExpectedCool':'cooling_setpoint',
    'TemperatureExpectedHeat':'heating_setpoint',
    'T_active':'setpoint'
})

# build timestamp
data['timestamp'] = pd.to_datetime(data['date'])
data['timestamp'] = data.apply(lambda x: x['timestamp'] + timedelta(hours=x['hour']), axis=1)
data['hour'] = data['timestamp'].dt.hour
data['day_of_year'] = data['timestamp'].dt.day_of_year
data['timestep'] = 24*(data['day_of_year'] - 1) + data['hour'] + 1

# convert to celcius
data['setpoint'] = pd.to_numeric(data['setpoint'], errors='coerce')
data['setpoint'] = (data['setpoint'] - 32)*5.0/9.0

# fill missing values
data = data.pivot(index=['location','building_name'],columns='timestep',values='setpoint')
data = data.fillna(method='ffill', axis=1,limit=1)
data = data.fillna(method='bfill', axis=1,limit=1)
data = data.reset_index(drop=False).melt(
    id_vars=['location','building_name'],
    value_vars=data.columns.tolist(),
    value_name='setpoint',
    var_name='timestep'
)

location_query = """
INSERT OR IGNORE INTO ecobee_location (name)
VALUES (:location)
;"""
location_values = data.groupby('location').size().reset_index().to_dict('records')

building_query = """
INSERT INTO ecobee_building (location_id, name)
VALUES ((SELECT id FROM ecobee_location WHERE name = :location), :building_name)
ON CONFLICT (name) DO UPDATE SET location_id = EXCLUDED.location_id
;"""
building_values = data.groupby(['building_name','location']).size().reset_index().to_dict('records')

timeseries_query = """
INSERT INTO ecobee_timeseries (building_id, timestep, setpoint)
VALUES ((SELECT id FROM ecobee_building WHERE name = :building_name), :timestep, :setpoint)
ON CONFLICT (building_id, timestep) DO UPDATE SET
    (setpoint) = (EXCLUDED.setpoint)
;"""
timeseries_values = data.to_dict('records')

DATABASE.insert_batch(
    [location_query, building_query, timeseries_query],
    [location_values, building_values, timeseries_values]
)